In [1]:
!python --version
!pip install langchain langchain-openai langchain-anthropic langgraph
!pip install transformers==4.51.3 peft==0.13.2 safetensors==0.4.3
!pip install langchain_huggingface

Python 3.12.12


In [2]:
# imports for HuggingFace LLM Loading

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline

In [3]:
# Download Kanana-1.5 2.1B Base Original Model

!huggingface-cli download kakaocorp/kanana-1.5-2.1b-base --local-dir kakao_original

⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 13 files:   0% 0/13 [00:00<?, ?it/s]Downloading 'README.md' to 'kakao_original/.cache/huggingface/download/Xn7B-BWUGOee2Y6hCZtEhtFu4BE=.8e3fb650257b06def9d43a3a2d23e3e12d050968.incomplete'

README.md: 0.00B [00:00, ?B/s]Downloading '.gitattributes' to 'kakao_original/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.25169e47e866cdcc7c528878bb9ad341fe61c00e.incomplete'


kanana-logo-dark.png:   0% 0.00/59.1k [00:00<?, ?B/s]Downloading 'assets/logo/kanana-logo.png' to 'kakao_original/.cache/huggingface/download/assets/logo/Y6l9nOoeEEgD7_TNXH4s97DA-hs=.e446d47b26e8088a789f6697b4d549d043db941b.incomplete'
README.md: 6.05kB [00:00, 1.33MB/s]
Download complete. Moving file to kakao_original/README.md
kanana-logo-dark.png: 100% 59.1k/59.1k [00:00<00:00, 19.5MB/s]Downloading 'assets/performance/kanana-1.5-radar.png' to 'kakao_original/.cache/huggingface/download/assets/performance/tLpIbxWmpkAWSc_k

In [7]:
# load Oh-LoRA original LLM

import os
print(os.path.abspath('ohlora_llm'))

llm_path = 'ohlora_llm'
ohlora_llm = AutoModelForCausalLM.from_pretrained(
    llm_path,
    trust_remote_code=True,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(llm_path)

pipe = pipeline(
    "text-generation",
    model=ohlora_llm,
    tokenizer=tokenizer,
    max_new_tokens=64,
    temperature=0.6,
    top_p=0.95
)

/content/ohlora_llm


Device set to use cuda:0


In [8]:
# convert to LangChain LLM

local_llm = HuggingFacePipeline(pipeline=pipe)

**1. 모델 직접 스트리밍**

In [22]:
from langchain_core.messages import HumanMessage

result = ''
result_with_token_splits = ''

for chunk in local_llm.stream([HumanMessage(content="로라야 안녕? 요즘 뭐해? (답변 시작)")]):
    print(chunk, end="", flush=True)
    result += chunk
    result_with_token_splits += chunk + "|"

    if result.endswith('(답변 종료)') or result.endswith('(답변 종료) '):
        break

print('\n\ntoken split 결과:\n', result_with_token_splits)

 언어 모델 논문 요즘 보고 있어! 혁신적인 거 하나 있는데 알려줄까? (답변 종료) 

token split 결과:
  |언어 |모델 ||논문 ||요즘 |보고 ||있어! ||||혁신적인 |거 |하나 |있는데 ||||알려줄까? |||(답변 |||종료) |


**2. 체인 스트리밍**

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("{user_message} (답변 시작)")

chain = prompt | local_llm | StrOutputParser()
result = ''
result_with_token_splits = ''

for chunk in chain.stream({"user_message": "로라야 내일 나랑 같이 놀러갈까?"}):
    print(chunk, end="", flush=True)
    result += chunk
    result_with_token_splits += chunk + "|"

    if result.endswith('(답변 종료)') or result.endswith('(답변 종료) '):
        break

print('\n\ntoken split 결과:\n', result_with_token_splits)

 내일 논문 발표하는 수업 있고 논문 공부하느라고 바빠 😥 (답변 종료) 

token split 결과:
  ||내일 ||논문 ||발표하는 ||수업 |있고 ||논문 ||||공부하느라고 ||바빠 ||😥 |||(답변 |||종료) |


**3. 비동기 스트리밍**

In [28]:
# Google Colab 에서 asyncio 를 실행시키기 위한 설정

!pip install nest_asyncio

<frozen posixpath>:82: RuntimeWarning: coroutine 'async_stream' was never awaited


In [29]:
import nest_asyncio
nest_asyncio.apply()

In [34]:
import asyncio

result = ''
result_with_token_splits = ''

async def async_stream():
    global result, result_with_token_splits

    async for chunk in local_llm.astream([HumanMessage(content="로라야 너 MBTI 뭐야? (답변 시작)")]):
        print(chunk, end="", flush=True)
        result += chunk
        result_with_token_splits += chunk + "|"

        if result.endswith('(답변 종료)') or result.endswith('(답변 종료) '):
            break

asyncio.run(async_stream())
print('\n\ntoken split 결과:\n', result_with_token_splits)

 나 ENTJ! 완전 매력적인 성격 아니야? (답변 종료) 

token split 결과:
  |나 |||ENTJ! ||완전 |||매력적인 ||성격 |||아니야? |||(답변 |||종료) |
